In [ ]:
import pandas
import theano.tensor as T
import numpy as np
import pymc3 as pm
from functools import reduce
from preprocessing import save_sequences, extract_amounts_on_board
import matplotlib.pyplot as plt
import math


fullCsvName = './data/alldata.csv'
binSize = 0.5

allStats = {}
segmentsToProcess = range(20,30)


for segment in segmentsToProcess:

    segmentCsvName = './processed/segment_' + str(segment) + '.csv'
    save_sequences(fullCsvName, segmentCsvName, b=binSize, d=3, segmentDays=30, segment=segment, 
                   sequenceLength=15, maxGapHours=16)

    df = pandas.read_csv(segmentCsvName)

    (basal, smbg, newCarbs, newInsulin, delT, insActCurve, carbActCurve,
            currentIOB, currentCOB, iobImpact, cobImpact, nSeq, nSeg) = extract_amounts_on_board(df, binSize)

    with pm.Model() as model:

        M = pm.Uniform('Basal', lower=0, upper=40) # Prior on basal metabolism
        CF = pm.Normal('CF', mu=250, sd=100) # Prior on correction factor
        CR = pm.Normal('CR', mu=40, sd=25) # Prior on correction factor
        sigmaBG = pm.Normal('SigmaBG', mu=50, sd=10) # Prior on measurement error, assuming
            # normally distributed. Note that log-normal distribution of glucose measurement 
            # would be more realistic, but a lot slower to run.
        # tauBG = 0.095; # SD of log(BG). If 95% obs w/i 20%, then SD ~ 10%. log(1.1) = 0.095.

        glus = []
        glus.append(pm.Normal('glucose0', mu=150, sd=80, observed=smbg[0]))

        # TODO: infer true IOB, COB

        for i in range(1, nSeg):
            print(i)
            # Calculate predicted next glucose value, based on...
            glus.append(pm.Normal('glucose' + str(i), 
                                  mu = glus[i-1] + # Last true glucose value
                                       -1 * np.dot(newInsulin[i-1], insActCurve) * CF + # Newly-added insulin
                                       -1 * np.transpose([np.dot(currentIOB[i-1][j], iobImpact[i][j]) for j in range(nSeq)]) * CF +   # All insulin, including new
                                       np.dot(newCarbs[i-1], carbActCurve) * CF / CR + # Newly-added carbs
                                       np.transpose([np.dot(currentCOB[i-1][j], cobImpact[i][j]) for j in range(nSeq)]) * CF / CR + # Carbs on board
                                       (M - basal[i-1]) * delT[i-1]/24 * CF, # Basal/metabolism mismatch
                                  sd = sigmaBG, 
                                  observed=smbg[i]))

        # Inference!
        trace = pm.sample(4000, tune=1000, progressbar=True) # draw posterior samples using NUTS sampling

    plt.figure(figsize=(7, 7))
    pm.traceplot(trace[100:])
    plt.tight_layout();

    stats = pm.summary(trace)
    print('Segment ', segment)
    print(stats)
    allStats[segment] = stats
    # print(stats.at['CF', 'mean']) # To access individual values

In [ ]:
allStats